In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import pandas as pd
import librosa
import logging
import os
from scipy.io import wavfile
import vae

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# Configure the logging settings
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [2]:
# Check available GPUs
physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices

[]

In [3]:
if len(physical_devices) > 0:
    # Ensure TensorFlow uses the first GPU
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    with tf.device('/GPU:0'):
        
else:
    print("No GPU devices found.")

IndentationError: expected an indented block (185336717.py, line 8)

In [3]:

# Step 1: Load, preprocess, and pad audio data
def preprocess_audio(audio_file, target_length=300):
    #logging.info(f'Processing audio file: {audio_file}')
    audio, _ = librosa.load(audio_file, sr=22050)
    audio_mfcc = librosa.feature.mfcc(y=audio, sr=22050)

    # Calculate the current length of audio data
    current_length = audio_mfcc.shape[1]

    if current_length < target_length:
        # If the audio is shorter than the target length, pad it
        pad_width = target_length - current_length
        audio_mfcc = np.pad(audio_mfcc, ((0, 0), (0, pad_width)), mode='constant')
    else:
        # If the audio is longer, truncate it
        audio_mfcc = audio_mfcc[:, :target_length]

    return audio_mfcc

# Generator function for lazy loading of audio data
def audio_data_generator(audio_files):
    for audio_file in audio_files:
        yield preprocess_audio(audio_file)

# Load a list of audio files
audio_files_df = pd.read_csv('data_files_summary.csv')
audio_files = audio_files_df['full_path'].to_list()

In [4]:
# Create a generator to load and preprocess audio data on-the-fly
data_generator = audio_data_generator(audio_files)
data_iterator = iter(data_generator)  # Convert the generator to an iterator

# To train an autoencoder, you need target data, which is the same as the input data
# So, use the same data for both input and target
X_train = np.array(list(data_iterator)).reshape(6300, 20, 300, 1)

In [37]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Normalization, Reshape, Dense, Flatten
from tensorflow.keras.models import Model

# Define the input shape
input_shape = (20, 300, 1)  # You can adjust the input shape based on your data

# Define the encoder
input_layer = Input(shape=input_shape)
x = input_layer

x = Normalization()(x)

# Convolutional encoder layers with variable filters
filters = [32, 64, 128]  # You can adjust the number of filters as needed
for num_filters in filters:
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

encoder = Model(input_layer, x, name='encoder')

# Define the decoder
latent_input = Input(shape=x.shape[1:])
x = latent_input

# Convolutional decoder layers with variable filters
for num_filters in filters[::-1]:
    x = Conv2DTranspose(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)

# Output layer
x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
x = Flatten()(x)
x = Dense(300 * 20 * 1, activation='relu')(x)  # Adjust units as needed
# Reshape to the desired 3D shape
output_layer = Reshape((20, 300, 1))(x)

decoder = Model(latent_input, output_layer, name='decoder')

# Create the variable convolutional autoencoder
autoencoder_input = Input(shape=input_shape)
encoded = encoder(autoencoder_input)
decoded = decoder(encoded)
autoencoder = Model(autoencoder_input, decoded, name='autoencoder')

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')
# Print the model summary
autoencoder.summary()


Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_39 (InputLayer)       [(None, 20, 300, 1)]      0         
                                                                 
 encoder (Functional)        (None, 3, 38, 128)        92675     
                                                                 
 decoder (Functional)        (None, 20, 300, 1)        44022129  
                                                                 
Total params: 44,114,804
Trainable params: 44,114,801
Non-trainable params: 3
_________________________________________________________________


In [ ]:
# Step 3: Train the autoencoder using audio data
logging.info('Training the autoencoder...')
autoencoder.fit(X_train, X_train, batch_size=32, epochs=300)
logging.info('Training complete.')

2023-10-19 20:52:25,290 - root - INFO - Training the autoencoder...


Train on 6300 samples
Epoch 1/300
 672/6300 [==>...........................] - ETA: 2:15 - loss: 5400.6788

In [26]:
from scipy.io import wavfile
# Step 4: Stream audio files through the trained autoencoder and save the encoded audio as WAV
def stream_audio_through_autoencoder(audio_file, autoencoder, output_path):
    input_audio = preprocess_audio(audio_file)
    encoded_audio = autoencoder.predict(np.expand_dims(input_audio, axis=0))
    
    # Inverse transform the encoded audio back to the waveform
    decoded_audio = librosa.feature.inverse.mfcc_to_audio(encoded_audio[0], sr=22050)
    
    # Scale the audio data to the appropriate range
    decoded_audio = (decoded_audio * np.iinfo(np.int16).max).astype(np.int16)
    
    # Save the decoded audio as a WAV file
    wavfile.write(output_path, 22050, decoded_audio)

# Specify the output path for the saved WAV file
output_path = "encoded_audio.wav"

# Process an audio file and save the encoded audio as a WAV
stream_audio_through_autoencoder("Recording (42).m4a", autoencoder, output_path)

1/1 [==============================] - ETA: 0s

C:\Users\gabri\AppData\Local\Temp\ipykernel_8936\1920379383.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(audio_file, sr=22050)
C:\Users\gabri\anaconda3\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 [==============================] - 0s 72ms/step
